In [2]:
import pandas as pd
from PIL import Image

train_labels = pd.read_csv('data/train.csv')

train_images= []

for img in train_labels['image']:
    image = Image.open(f'data/train/{img}')
    train_images.append(image)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\HP\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_27444\460305767.py", line 4, in <module>
    train_labels = pd.read_csv('data/train.csv')
  File "c:\Python39\lib\site-packages\pandas\io\parsers\readers.py", line 1026, in read_csv
  File "c:\Python39\lib\site-packages\pandas\io\parsers\readers.py", line 620, in _read
  File "c:\Python39\lib\site-packages\pandas\io\parsers\readers.py", line 1620, in __init__
  File "c:\Python39\lib\site-packages\pandas\io\parsers\readers.py", line 1880, in _make_engine
  File "c:\Python39\lib\site-packages\pandas\io\common.py", line 873, in get_handle
OSError: [Errno 24] Too many open files: 'data/train.csv'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\HP\AppData\Roaming\Python\Python39\site-packages\IPython\core\interact

In [ ]:
test_labels = pd.read_csv('data/test.csv')
test_images= []

for img in test_labels['image']:
    image = Image.open(f'data/test/{img}')
    test_images.append(image)

In [ ]:
val_labels = pd.read_csv('data/validation.csv')
val_images= []

for img in val_labels['image']:
    image = Image.open(f'data/validation/{img}')
    val_images.append(image)